In [2]:
# Imports used the the project

# Display plots directly in the notebook instead of in a new window
%matplotlib inline

# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Configure libraries
# The seaborn library makes plots look nicer
sns.set()
sns.set_context('talk')

# Don't display too many rows/cols of DataFrames
pd.options.display.max_rows = 7
pd.options.display.max_columns = 8

# Round decimals when displaying DataFrames
pd.set_option('precision', 2)

# Data Wrangling

In [11]:
df_airports = pd.read_csv("Airports.csv")
df_airports

,Origin_airport,Destination_airport,Origin_city,Destination_city,...,Distance,Fly_date,Origin_population,Destination_population
0,MHK,AMW,"Manhattan, KS","Ames, IA",...,254,200810,122049,86219
1,EUG,RDM,"Eugene, OR","Bend, OR",...,103,199011,284093,76034
2,EUG,RDM,"Eugene, OR","Bend, OR",...,103,199012,284093,76034
...,...,...,...,...,...,...,...,...,...
3606800,STL,TBN,"St. Louis, MO","Fort Leonard Wood, MO",...,119,200908,2828990,46457
3606801,CGI,TBN,"Cape Girardeau, MO","Fort Leonard Wood, MO",...,146,200908,93712,46457
3606802,FWA,OH1,"Fort Wayne, IN","Washington Court House, OH",...,135,200309,398574,28133


Los Angeles

In [5]:
df_la = pd.read_csv('./WEATHERDATA/LA_PL.csv')
df_la = df_la.drop(columns = "Unnamed: 0")
df_la.head() #Pressure ('Hg) needs to be renamed ("Hg)

,Temperature (F),Humidity (%),Pressure ('Hg)
0,60,0.59,30.07
1,58,0.53,30.04
2,59,0.71,30.03
3,62,0.64,29.99
4,63,0.70,29.95


In [12]:
New York City

SyntaxError: invalid syntax (<ipython-input-12-cba9da2dbdb2>, line 1)

In [6]:
df_nyc = pd.read_csv('./WEATHERDATA/NYC_PL.csv')
df_nyc.head()

,,,NYC_PL
NaN,Temperature (F),Humidity (%),"Pressure (""Hg)"
0.0,33,0.62,30.16
1.0,42,0.71,30.19
2.0,39,0.61,29.89
3.0,49,0.6,29.98


We are going to need to clean this

Chicago

In [8]:
df_ch = pd.read_csv('./WEATHERDATA/CH_PL.csv')
df_ch = df_ch.drop(columns = 'Unnamed: 0')
df_ch.head()

,Temperature (F),Humidity (%),"Pressure (""Hg)"
0,27,0.74,30.20
1,31,0.76,30.16
2,37,0.63,30.10
3,42,0.61,30.04
4,66,0.68,29.94


Seattle

In [9]:
df_sea = pd.read_csv('./WEATHERDATA/SEA_PL.csv')
df_sea = df_sea.drop(columns = 'Unnamed: 0')
df_sea.head()

,Temperature (F),Humidity (%),"Pressure (""Hg)"
0,45,0.85,30.03
1,42,0.77,30.16
2,46,0.73,30.01
3,51,0.72,30.02
4,60,0.65,30.05


Orlando

In [15]:
df_orl = pd.read_csv('./WEATHERDATA/ORL_PL.csv')
df_orl = df_orl.drop(columns = 'Unnamed: 0')
df_orl.head()

,Temperature (F),Humidity (%),"Pressure (""Hg)"
0,59,0.72,30.20
1,71,0.77,30.24
2,66,0.58,30.06
3,73,0.68,30.05
4,76,0.80,30.03
